In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
clv = True
NUM_THREAD = 5
STATUS = {0:"commit", 2:"abort"}
def get_thd(txn):
    return int(txn) % NUM_THREAD
def get_lock(lock_idx):
    lock_idx = int(lock_idx)
    if lock_idx == 0:
        return "SH"
    elif lock_idx == 1:
        return "EX"
    else:
        return "NONE"
def rm_txn(row, place, txn):
    thread = get_thd(txn)
    members = locktable[row][place]
    found = False
    for i, step in enumerate(members):
        if step[0] == txn:
            found = True
            break
    if not found:
        print("not found: {}".format(line))
        raise
    locktable[row][place] = members[:i] + members[i+1:]
    return thread, members[i]
def get_schedule(start, end, thread):
    return schedule.loc[start:end+1, thread][schedule.loc[start:end+1, thread].notnull()]
def find_locktable_with_txn(txn):
    for row in locktable:
        if txn in [item[0] for place in ['owners', 'waiters', 'retired'] for item in locktable[row][place] ]:
            print(row)
            print(locktable[row])
def find_locktable_with_row(row):
    print(locktable[row])
def get_p(txn):
    if txn in priority:
        p = priority[txn]
    else:
        p = -1
    return p

In [16]:
def parse(line):
    txn = ""
    tokens = line.split(" ")
    if "[thread]" in line:
        if "start running txn" in line:
            thread = int(tokens[2])
            txn = tokens[6]
            schedule.loc[ts, thread] = "start txn %s"%txn
            if clv:
                barriers[txn] = 0
        elif "finish txn" in line:
            txn = tokens[3]
            thread = get_thd(txn)
            rc = tokens[-1]
            # "[thread-%lu] finish txn %lu (aborted)\n"
            schedule.loc[ts, thread] = "thread finish txn %s %s"%(txn, thread)
            
    elif "[row_" in tokens[0]:
            #return
            if "add txn" in line:
                    #[row_ww] add txn 1 type 1 to owners of row 1449388428864366
                    txn = tokens[3]
                    lock = get_lock(tokens[5])
                    row = tokens[-1]
                    thread = get_thd(txn)
                    if row not in locktable:
                        locktable[row] = {}
                        locktable[row]['owners'] = []
                        locktable[row]['waiters'] = []
                        if clv:
                            locktable[row]['retired'] = []
                    
                    locktable[row][tokens[7]].append((txn, get_p(txn), lock))
                    if "owner" in tokens[7]:
                        schedule.loc[ts, thread] = "acquire %s lock %s"%(lock, row)
                    elif "waiter" in tokens[7]:
                        schedule.loc[ts, thread] = "wait %s lock %s"%(lock, row)
                    elif "retired" in tokens[7]:
                        schedule.loc[ts, thread] = "retire %s lock %s"%(lock, row)
            elif "rm another txn" in line:
                    # [row_clv] txn %lu rm another txn %lu from owners of row %lu
                    wounder = tokens[2]
                    txn = tokens[6]
                    row = tokens[-1]
                    place = tokens[8]
                    thread, _ = rm_txn(row, place, txn)
                    if "owner" in place:
                        schedule.loc[ts, thread] = "removed lock %s from owners by %s"%(row, wounder)
                    elif "retired" in place:
                        schedule.loc[ts, thread] = "removed lock %s from retired by %s"%(row, wounder)
            elif "rm txn" in line:
                    txn = tokens[3]
                    row = tokens[-1]
                    thread, _ = rm_txn(row, tokens[5], txn)
                    if "owner" in tokens[5]:
                        schedule.loc[ts, thread] = "remove lock %s from owners"%row
                    elif "waiter" in tokens[5]:
                        schedule.loc[ts, thread] = "remove lock %s from waiters"%row
                    elif "retired" in tokens[5]:
                        schedule.loc[ts, thread] = "release lock %s from retired"%row
            elif "from waiter to owner" in line or "from waiters to owners" in line:
                    # [row_ww] bring 0 from waiter to owner of row 1454103086539417
                    txn = tokens[2]
                    row = tokens[-1]
                    thread = get_thd(txn)
                    found = False
                    thread, entry = rm_txn(row, "waiters", txn)
                    entry = (entry[0], get_p(txn), entry[-1])
                    locktable[row]["owners"].append(entry)
                    schedule.loc[ts, thread] = "acquire lock %s"%row
            elif "from owners to retired" in line:
                    # [row_clv] move txn 0 from owners to retired type 0 of row 1459774078579136
                    txn = tokens[3]
                    row = tokens[-1]
                    thread = get_thd(txn)
                    found = False
                    lock = get_lock(tokens[-4])
                    locktable[row]["retired"].append((txn, get_p(txn), lock))
                    schedule.loc[ts, thread] = "retire lock %s"%row
            elif "abort txn" in line:
                # [row_ww] txn 8 abort txn 6 on row 10
                wound = tokens[2]
                abort = tokens[-4]
                row = tokens[-1]
                txn = wound
                schedule.loc[ts, get_thd(wound)] = "wound %s on %s"%(abort, row)
                schedule.loc[ts, get_thd(abort)] = "%s wounded by %s"%(abort,wound)
            elif "rm aborted txn" in line:
                # [row_clv] rm aborted txn %lu from retired of row %lu\n
                txn = tokens[4]
                row = tokens[-1]
                thread, _ = rm_txn(row, "retired", txn)
                schedule.loc[ts, thread] = "release lock %s from retired"%row
    elif "[txn]" in tokens[0]:
        if "set txn" in line and "to abort" in line:
            wounder = tokens[1]
            txn = tokens[4]
            schedule.loc[ts, get_thd(txn)] = "txn %s set abort by %s"% (txn, wounder)
            schedule.loc[ts, get_thd(wounder)] = "wound txn %s"%txn
        elif "set ts" in line:
            #[txn] set ts %lu for txn %lu\n
            txn = tokens[-1]
            p = tokens[3]
            if txn in priority:
                print("change priority of txn %s from %s to %s"%(txn, priority[txn], p))
            else:
                print("set txn %s ts %s"%(txn, p))
            priority[txn] = p
            schedule.loc[ts, get_thd(txn)] = "set ts %s"%p
        elif "is set to" in line:
            # [txn] txn %lu is set to commited
            txn = tokens[2]
            status = tokens[-1]
            thread = get_thd(txn)
            schedule.loc[ts, thread] = "txn %s set %s"%(txn, status) 
            print("txn %s set %s"%(txn, status) )
            if status != "commited":
                barriers[txn] = -1
        elif "barrier" in line:
            # "[txn] increment barrier for txn %lu\n"
            txn = tokens[-1]
            thread = get_thd(txn)
            if tokens[1] == "increment":
                barriers[txn] += 1
            elif tokens[1] == "decrement":
                barriers[txn] -= 1
            schedule.loc[ts, thread] = "txn %s %s barrier, now should be %d"%(txn, tokens[1], barriers[txn])
        elif "# bariers" in line:
            # [txn] # bariers of txn 1 = 0
            txn = tokens[-3]
            thread = get_thd(txn)
            schedule.loc[ts, thread] = "pre-commit: txn %s's barriers = %d"%(txn, int(tokens[-1]))
            assert(int(tokens[-1]) == barriers[txn])
    if "txn 3" in line or "rm" in line :
        print("[{}]: {}".format(ts, line))

In [17]:
output = open('debug-1.out')

In [18]:
locktable = {} # row_id: { owners: {}, waiters: {}, (opt) retired: {}}
barriers = {}
priority = {}
schedule = pd.DataFrame()
ts = 0

In [19]:
for line in tqdm(output):
    try:
        parse(line.strip())
        ts += 1
    except:
        print(line)
        raise

3521it [00:00, 34829.30it/s]

[89]: There is NO WARRANTY, to the extent permitted by law.  Type "show copying"
[175]: [Inferior 1 (process 38277) exited normally]
[214]: There is NO WARRANTY, to the extent permitted by law.  Type "show copying"
[300]: [Inferior 1 (process 38871) exited normally]
[376]: [Inferior 1 (process 38906) exited normally]
[452]: [Inferior 1 (process 38934) exited normally]
[499]: command 'srun' from deb slurm-client
[512]: There is NO WARRANTY, to the extent permitted by law.  Type "show copying"
[599]: [Inferior 1 (process 39801) exited normally]
[737]: There is NO WARRANTY, to the extent permitted by law.  Type "show copying"
[823]: [Inferior 1 (process 41699) exited normally]
[899]: [Inferior 1 (process 41730) exited normally]
[1162]: There is NO WARRANTY, to the extent permitted by law.  Type "show copying"
[1248]: [Inferior 1 (process 45227) exited normally]
[1324]: [Inferior 1 (process 45258) exited normally]
[1400]: [Inferior 1 (process 45286) exited normally]
[1476]: [Inferior 1 (pr

[3876]: [txn] increment barrier for txn 3
[3877]: [row_clv] move txn 3 from owners to retired type 0 of row 4355290486800249
[3878]: [row] txn 3 try to get row 4355290486801182
[3883]: [row_clv] rm txn 16 from owners of row 4355290486803810
[3884]: [row_clv] rm txn 14 from owners of row 4355290486822658
[3885]: [row_clv] rm txn 1 from owners of row 4355290486892877
[3887]: [row_clv] add txn 3 type 0 to waiters of row 4355290486801182
[3892]: [row_clv] rm txn 1 from owners of row 4355290486798084
[3895]: [row_clv] rm txn 3 from owners of row 4355290486801182
[3896]: [txn] increment barrier for txn 3
[3897]: [row_clv] move txn 3 from owners to retired type 0 of row 4355290486801182
[3898]: [row] txn 3 try to get row 4355290487779462
[3899]: [row_clv] add txn 3 type 1 to waiters of row 4355290487779462
txn 16 set aborted
[3905]: [row_clv] rm txn 3 from owners of row 4355290487779462
[3906]: [row_clv] move txn 3 from owners to retired type 1 of row 4355290487779462
[3907]: [row] txn 3 try 

AssertionError: 

In [7]:
import numpy as np
for i in schedule:
    print("thread {}: {}".format(i, schedule[i][schedule[i].notnull()].values[-1]))

thread 9: start txn 29
thread 4: retire lock 4355290486803810
thread 6: start txn 26
thread 1: retire lock 4355290486803810
thread 3: release lock 4355290486800249 from retired
thread 7: start txn 17
thread 8: start txn 28
thread 2: pre-commit: txn 17's barriers = 2
thread 0: remove lock 4355290486822658 from owners
thread 5: start txn 25


In [8]:
find_locktable_with_txn('9')

In [ ]:
# for row in get_schedule(10454, 10619, [0,1,2]).iterrows():
#     v = " | ".join([str(x) for x in row[1].values])
#     if "115690" in v or "160397" in v:
#         print(v)

In [ ]:
#schedule.to_csv('sample_schedule.csv', index=False)

In [9]:
for b in barriers:
    if barriers[b] > 0:
        print("{} = {} at thread {}".format(b, barriers[b], int(b)%NUM_THREAD))

17 = 1 at thread 2
22 = 1 at thread 2
24 = 1 at thread 4
26 = 1 at thread 1


In [10]:
schedule

,9,4,6,1,3,7,8,2,0,5
3459,start txn 9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3464,NaN,wait SH lock 4355290486822658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3472,NaN,NaN,start txn 6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3474,NaN,start txn 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3475,NaN,acquire lock 4355290486822658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3479,NaN,NaN,NaN,start txn 1,NaN,NaN,NaN,NaN,NaN,NaN
3481,NaN,NaN,NaN,NaN,start txn 3,NaN,NaN,NaN,NaN,NaN
3484,NaN,NaN,NaN,NaN,NaN,start txn 7,NaN,NaN,NaN,NaN
3486,NaN,NaN,NaN,NaN,NaN,NaN,start txn 8,NaN,NaN,NaN
3491,NaN,remove lock 4355290486822658 from owners,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
locktable

{'4355290486822658': {'owners': [],
  'waiters': [('20', -1, 'SH'), ('11', -1, 'SH')],
  'retired': [('3', -1, 'SH'),
   ('17', -1, 'SH'),
   ('24', -1, 'SH'),
   ('26', -1, 'SH'),
   ('22', -1, 'SH')]},
 '4355290486803810': {'owners': [],
  'waiters': [('22', -1, 'EX')],
  'retired': [('3', -1, 'EX'),
   ('17', -1, 'EX'),
   ('24', -1, 'EX'),
   ('26', -1, 'EX')]},
 '4355290486833157': {'owners': [],
  'waiters': [],
  'retired': [('3', -1, 'SH'), ('17', -1, 'SH')]},
 '4355290486800249': {'owners': [],
  'waiters': [],
  'retired': [('17', -1, 'SH')]},
 '4355290486801182': {'owners': [],
  'waiters': [],
  'retired': [('17', -1, 'SH')]},
 '4355290487779462': {'owners': [],
  'waiters': [],
  'retired': [('17', -1, 'EX')]},
 '4355290486806330': {'owners': [],
  'waiters': [],
  'retired': [('17', -1, 'SH')]},
 '4355290486626377': {'owners': [],
  'waiters': [],
  'retired': [('17', -1, 'SH')]},
 '4355290487570867': {'owners': [],
  'waiters': [],
  'retired': [('17', -1, 'SH')]},
 '435